In [1]:
import tensorflow as tf
import time

In [5]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # Opening the file
        time.sleep(0.03)

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)

            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )


In [6]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

In [7]:
%%timeit
benchmark(FileDataset())


388 ms ± 26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
benchmark(FileDataset().prefetch(1))

327 ms ± 22.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

343 ms ± 15.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.map(lambda x: x**2)
dataset = dataset.cache("mycache.txt")
# The first time reading through the data will generate the data using
# `range` and `map`.
list(dataset.as_numpy_iterator())


[0, 1, 4, 9, 16]

In [11]:
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [12]:
def mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s


In [13]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function), 5)

1.51 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

545 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
